In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
 ! pip install --upgrade kaggle -q

In [2]:
import re
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
from sklearn.model_selection import train_test_split
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D


import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from plotly import graph_objs as go

%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import files

In [5]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list
! kaggle competitions download -c 'jigsaw-multilingual-toxic-comment-classification' -p 'dataset' -q

Saving kaggle.json to kaggle.json


In [6]:
!unzip '/content/dataset/jigsaw-toxic-comment-train.csv.zip'
!unzip '/content/dataset/test.csv.zip'
!unzip '/content/dataset/validation.csv.zip'

Archive:  /content/dataset/jigsaw-toxic-comment-train.csv.zip
  inflating: jigsaw-toxic-comment-train.csv  
Archive:  /content/dataset/test.csv.zip
  inflating: test.csv                
Archive:  /content/dataset/validation.csv.zip
  inflating: validation.csv          


In [4]:
import tensorflow as tf

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.58.215.82:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.58.215.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.58.215.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [5]:
import pandas as pd
train = pd.read_csv('/content/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/content/validation.csv')
test = pd.read_csv('/content/validation.csv')

In [6]:
print(f"train shape: {train.shape} \nvalidation shape: {validation.shape} \ntest shape: {test.shape}")

train shape: (223549, 8) 
validation shape: (8000, 4) 
test shape: (8000, 4)


In [7]:
for col in train.columns:
  if (col not in validation.columns) or (col not in test.columns):
    print(col)

severe_toxic
obscene
threat
insult
identity_hate


In [8]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [29]:
def clean_text(text):
  text = re.sub(r'[^\w\s]','',text, re.UNICODE)
  text = text.replace('\n', ' ')
  text = text.lower()
  return text
  
%time train['clean_text'] = train.comment_text.apply(lambda x: clean_text(x))
%time test['clean_text'] = test.comment_text.apply(lambda x: clean_text(x))
%time validation['clean_text'] = validation.comment_text.apply(lambda x: clean_text(x))
train.head()

CPU times: user 131 ms, sys: 825 µs, total: 132 ms
Wall time: 134 ms


,id,comment_text,toxic,clean_text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,daww he matches this background colour im seem...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,hey man im really not trying to edit war its j...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,more i cant make any real suggestions on impr...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [11]:
mean_word_len = train.clean_text.apply(lambda x: len(x.split(" "))).mean()
print(f"Dataset with shape of {train.shape[0]} samples. \nMean number of words is: {mean_word_len}. \nDistribution of lables is: \n{train.toxic.value_counts()}")

Dataset with shape of 223549 samples. 
Mean number of words is: 70.65667482296946. 
Distribution of lables is: 
0    202165
1     21384
Name: toxic, dtype: int64


In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, 
                                                  stratify=train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [12]:
! pip install transformers -q

In [13]:
# Loading Dependencies
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers

from tokenizers import BertWordPieceTokenizer

In [14]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * 4 * strategy.num_replicas_in_sync
MAX_LEN = 200

In [15]:
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)

In [17]:
def convert_example_to_feature(text):
  
  # combine step for tokenization, WordPiece vector mapping, adding special tokens as well as truncating reviews longer than the max length
  
  return tokenizer.encode_plus(text, 
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = MAX_LEN, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
                return_token_type_ids = True,
                truncation=True
              )
# map to the expected input to TFBertForSequenceClassification, see here 
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(texts, lables, limit=-1):

  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []
    
  for text, label in tqdm(zip(texts, lables)):
    bert_input = convert_example_to_feature(text)
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])

  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

In [18]:
# train dataset
ds_train_encoded = encode_examples(train.clean_text.astype(str), train.toxic).shuffle(10000).batch(BATCH_SIZE)

# validation dataset
ds_valid_encoded = encode_examples(validation.clean_text.astype(str), validation.toxic).batch(BATCH_SIZE)

# test dataset
ds_test_encoded = encode_examples(test.clean_text.astype(str), test.toxic).batch(BATCH_SIZE)

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5

# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 2

223549it [04:59, 747.55it/s]
8000it [00:10, 730.21it/s]


In [22]:
# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased')

# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

my_callbacks = [
                tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='min', baseline=None, restore_best_weights=True)
]
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  167356416 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 167,357,954
Trainable params: 167,357,954
Non-trainable params: 0
_________________________________________________________________


In [24]:
bert_history = model.fit(ds_train_encoded, 
                         epochs=number_of_epochs, 
                         validation_data=ds_valid_encoded,
                         callbacks=my_callbacks)

Epoch 1/2
437/437 [==============================] - 36609s 84s/step - loss: 0.1217 - accuracy: 0.9524 - val_loss: 0.6315 - val_accuracy: 0.8497
Epoch 2/2
437/437 [==============================] - 35930s 82s/step - loss: 0.0873 - accuracy: 0.9645 - val_loss: 0.6951 - val_accuracy: 0.8500


In [25]:
model.save_pretrained('/content/drive/My Drive/projects/Jigsaw Multilingual Toxic Comment Classification/bert_model')
tokenizer.save_pretrained('/content/drive/My Drive/projects/Jigsaw Multilingual Toxic Comment Classification/bert_model')

('/content/drive/My Drive/projects/Jigsaw Multilingual Toxic Comment Classification/bert_model/vocab.txt',
 '/content/drive/My Drive/projects/Jigsaw Multilingual Toxic Comment Classification/bert_model/special_tokens_map.json',
 '/content/drive/My Drive/projects/Jigsaw Multilingual Toxic Comment Classification/bert_model/added_tokens.json')

In [26]:
new_model = TFBertForSequenceClassification.from_pretrained('/content/drive/My Drive/projects/Jigsaw Multilingual Toxic Comment Classification/bert_model')
new_tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/projects/Jigsaw Multilingual Toxic Comment Classification/bert_model')

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
new_model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  167356416 
_________________________________________________________________
dropout_113 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 167,357,954
Trainable params: 167,357,954
Non-trainable params: 0
_________________________________________________________________


In [30]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
new_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

new_model.evaluate(ds_test_encoded)

8000it [00:10, 736.98it/s]


16/16 [==============================] - 455s 28s/step - loss: 0.6951 - accuracy: 0.8500


[0.6951270699501038, 0.8500000238418579]